In [1]:
import osmnx as ox
from streamlit_functions import get_nodes_with_tags_in_bbox, count_tag_frequency, gdf_data

In [2]:
places = ["Kreuzberg"]
places_gdf = ox.geocode_to_gdf(places)
# bbox = [S, W, N, E]
bounding_boxes = places_gdf.loc[:, ["bbox_south", "bbox_west", "bbox_north", "bbox_east",]]
places_gdf

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((13.36823 52.49334, 13.36917 52.49315...",52.509382,52.482792,13.45293,13.368229,307579131,relation,55765,52.497644,13.411914,"Kreuzberg, Friedrichshain-Kreuzberg, Berlin, G...",boundary,administrative,0.628946


In [5]:
import json
keys = []
nodes = []
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    unique_tags_dict = count_tag_frequency(nodes)
    num_unique_values = {k:len(v) for k, v in unique_tags_dict.items()}
    num_unique_values = {
        k: v
        for k, v in sorted(
            num_unique_values.items(), key=lambda item: item[1], reverse=True
        )
    }

unique_tags_dict_sorted = [unique_tags_dict[k] for k in list(num_unique_values.keys())]

num_unique_values
    

{'name': 4954,
 'contact': 2432,
 'opening_hours': 2012,
 'website': 1560,
 'addr': 1417,
 'ref': 992,
 'phone': 779,
 'operator': 734,
 'check_date': 552,
 'brand': 474,
 'description': 420,
 'image': 281,
 'email': 260,
 'note': 230,
 'wikidata': 203,
 'mapillary': 202,
 'memorial': 184,
 'direction': 170,
 'wheelchair': 152,
 'inscription': 144,
 'seamark': 137,
 'shop': 131,
 'official_ref': 124,
 'railway': 123,
 'cuisine': 122,
 'survey': 119,
 'wikimedia_commons': 116,
 'amenity': 116,
 'disused': 109,
 'camera': 107,
 'wikipedia': 102,
 'collection_times': 102,
 'start_date': 89,
 'source': 79,
 'healthcare': 66,
 'playground': 64,
 'fixme': 62,
 'capacity': 60,
 'artist_name': 56,
 'maxheight': 53,
 'fax': 52,
 'alt_name': 49,
 'office': 46,
 'subject': 42,
 'short_name': 41,
 'dance': 41,
 'removed': 39,
 'craft': 38,
 'river': 34,
 'sport': 33,
 'bicycle_parking': 33,
 'ele': 31,
 'height': 31,
 'diameter_crown': 31,
 'old_name': 30,
 'genus': 30,
 'access': 29,
 'obstacle_n

In [7]:
response_json = "{\n  \"place\": \"Monbijoupark\"\n}"

import sys, json
struct = {}


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2822234876.py, line 10)

In [ ]:

dict(zip(places_gdf["display_name"], places_gdf["projected_area"]))

In [ ]:
places_gdf[["projected_area", "area_unit"]] = places_gdf.apply(lambda row: gdf_data(row, places_gdf.crs), axis=1)
places_gdf

In [ ]:
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    keys = list(count_tag_frequency(nodes).keys())
keys

In [ ]:
import utm
from pyproj import CRS
import geopandas as gpd
def gdf_data(gdf):
    """Get the area of a polygon
    This method is not directly callable by the LLM"""
    places_dict = {}
    for index, row in gdf.iterrows():
        print(index)
        utm_zone = utm.latlon_to_zone_number(gdf.loc[[index], "lat"].values[0], gdf.loc[[index], "lon"].values[0])
        south = gdf.loc[[index], "lat"].values[0] < 0
        crs = CRS.from_dict({"proj": "utm", "zone": utm_zone, "south": south})
        epsg_code = crs.to_authority()[1]
        unit = list({ai.unit_name for ai in crs.axis_info})[0]
        gdf_projected = gdf.loc[[index],:].to_crs(epsg_code)
        area = gdf_projected.area.values[0]
        places_dict[row["display_name"]] = {"area":area,
                                            "unit":unit}

    return places_dict

In [ ]:
gdf_data(places_gdf)